# Order a Basemap Using the Orders API via Quad ID
---

## Overview ##
---
Download Basemap quads based on the mosaic name and quad IDs.

For example:
point_reyes_rolling_normalized_2020-04-30_mosaic

and

325-1260,326-1260,325-1259,326-1259


<a name="Get-quad-IDs"></a> 
## Get quad IDs
---

You can get a basemap ID from Planet Explorer or the Basemap Viewer, as described in jupyter-notebooks/quad_tutorial/basemaps_ordersv2_aoi.ipynb. Each basemap is broken down into quads. To get the quad IDs for a basemap:

1. In the Basemap Viewer, enter your basemap ID, such as point_reyes_rolling_normalized_2020-04-30_mosaic
2. On the resulting basemap, select the arrow to view the quads and scenes, for example: 325-1260,326-1260,325-1259,326-1259


<a name="how-to-get-started"></a> 
## Start coding

---
Next set up a session by importing needed Python packages, pulling in your API Key, and make an initial request (to retrieve the Orders API parameters) to confirm a connection with the server.

In [ ]:
import planet
import os
import requests
from requests.auth import HTTPBasicAuth
import json
import time


<a name="setup-session"></a> 
## Setup Session Using Orders API
---
The first step is to setup a session to access the Orders API. The Orders API can be used to download full mosaics, or to download individual parts of the mosaic known as quads. 

### Create Session to Access API
---
Use your Planet API key for successful authentication. You can retrieve it from your accounts dashboard or in Explorer when selecting imagery. Save your file as an environmental variable. Here, the API key is saved to a zsh file and imported into the python script.

In [ ]:
ORDERS_API_URL = 'https://api.planet.com/compute/ops/orders/v2'
API_KEY = os.environ.get('PL_API_KEY')
GCP_CREDENTIALS = os.environ.get('GCP_CREDENTIALS')
orders_url = 'https://api.planet.com/compute/ops/orders/v2'
auth = HTTPBasicAuth(API_KEY, '')


# Setup the session
session = requests.Session()

# Authenticate
session.auth = (API_KEY, "")


### Send Request and Check Status Code
---
When sending the request, we will provide one parameter. Assuming we know the name of the mosaic, we can include those details in the request to access the corresponding metadata. After sending the request, let's check the status code to ensure it was successful.

Send a get request and print out the response, which should return `<Response [200]>`.

In [ ]:
response = requests.get(orders_url, auth=session.auth)
response


## Use Basemaps API to get the Basemap and quad IDs
---

Use the Basemap APIs to retrieve the basemap you want and the quad IDs. You can get the base URL you'll need to communicate with the Basemap API service:

1. Go to the [Basemap API reference](https://developers.planet.com/docs/basemaps/reference/) at https://developers.planet.com/docs/basemaps/reference/. 
2. Under List Mosaics, select GET /mosaics.

A dropdown UI element appears with the URL to use:

https://api.planet.com/basemaps/v1/mosaics


#### Search for the mosaic ID to get its quad IDs

Here, we are using the full name of the mosaic, but with the `name__contains` field, you can use a partial name based on the names of the mosaics available through your account.

Declare the search string for `requests.get()`. You will use the dictionary you create with the mosaic name to pass to the requests.get() function.


In [ ]:
BASEMAP_API_URL = 'https://api.planet.com/basemaps/v1/mosaics'
headers = {'content-type': 'application/json'}

basemap_params = {
    'name__contains': 'point_reyes_rolling_normalized_2020-04-30_mosaic'
}

basemapServiceResponse = requests.get(
    url=BASEMAP_API_URL, params=basemap_params, auth=auth, headers=headers)

basemaps = basemapServiceResponse.raise_for_status()
if basemapServiceResponse.status_code != 204:
    basemaps = json.loads(basemapServiceResponse.text)
print(basemaps)


In [ ]:
mosaicID = basemaps['mosaics'][0]['id']
mosaicBbox = basemaps['mosaics'][0]['bbox']
mosaicBboxStr = ','.join(map(str, mosaicBbox))

quad_params = {
    'mosaic_id': mosaicID,
    'bbox': mosaicBboxStr,
}

quads_url = "{}/{}/quads".format(BASEMAP_API_URL, mosaicID)
quadServiceResponse = requests.get(
    url=quads_url, params=quad_params, auth=auth, headers=headers)
quads = quadServiceResponse.json()
items = quads['items']

quadIds = []
for i in items:
    quadId = i['id']
    quadIds.append(quadId)


### Create an order packet
---
Package up the details of your order in a JSON object and make a POST request, passing in the Orders URL, your JSON, your API key, and the content-type header. 

In [ ]:
order_params = {
    "name": "basemap order with quad_ids",
    "source_type": "basemaps",
    "products": [
        {
            "mosaic_name": "point_reyes_rolling_normalized_2020-04-30_mosaic",
            "quad_ids": quadIds
        }
    ],
    "tools": [
        {
            "reproject": {
                "projection": "EPSG:4326",
                "resolution": 3.0 / 111000.0,
                "kernel": "cubic"
            }
        },
        {
            "bandmath": {
                "b1": "b1",
                "b2": "b2",
                "b3": "b3",
                "b4": "arctan(b1)",
                "b5": "(b4-b3)/(b4+b3)",
                "pixel_type": "32R"
            }

        }

    ],


    "delivery": {
        "google_cloud_storage": {
            "bucket": "notebook-practice",
            "credentials": GCP_CREDENTIALS,
            "path_prefix": "basemaps-to-cloud/",
        }
    }
}


### Send Request and Check Status Code
---
When sending the request, we will provide one parameter. Assuming we know the name of the mosaic, we can include those details in the request to access the corresponding metadata. After sending the request, let's check the status code to ensure it was successful.

Send a get request and print out the response, which should return `<Response [200]>`.

In [ ]:
paramRes = requests.post(ORDERS_API_URL, data=json.dumps(
    order_params), auth=auth, headers=headers)

paramRes.raise_for_status()
if paramRes.status_code != 204:
    print(paramRes.text)


### Place the order
---
The following code  

In [ ]:
def place_order(order_params, auth):
    response = requests.post(ORDERS_API_URL, data=json.dumps(
        order_params), auth=auth, headers=headers)
    print(response)
    order_id = response.json()['id']
    print(order_id)
    order_url = ORDERS_API_URL + '/' + order_id
    return order_url


order_url = place_order(order_params, session.auth)


### Poll for data readiness
---
When you're ready to pull the data, you can poll for when it's ready. Here, the polling is limited to a number of loops routine.

In [ ]:
def poll_for_success(order_url, auth, num_loops=30):
    count = 0
    while(count < num_loops):
        count += 1
        r = requests.get(order_url, auth=session.auth)
        response = r.json()
        state = response['state']
        print(state)
        end_states = ['success', 'failed', 'partial']
        if state in end_states:
            break
        time.sleep(10)


poll_for_success(order_url, auth)


In [ ]:
r = requests.get(order_url, auth=session.auth)
response = r.json()
results = response['_links']['results']


In [ ]:
[r['name'] for r in results]
